In [1]:
import pandas as pd
from glob import glob
import json
import os

In [61]:
def get_filelst(SAVE_PATH):
    return list(set([i.replace("_0_shot.json","") for i in os.listdir(SAVE_PATH)]))

# results/korean_origin_bench/20b/00_shot.json
def _get_metric_name(v):
    metrics = ['f1', 'macro_f1', 'acc_norm', 'acc']
    for m in metrics:
        if v.get(m):
            return {
                'metric': m,
                'value': v[m],
            }

def get_df_klue(SAVE_PATH, model_name=''):
    
    data = []
    for i in ['0', '5', '10', '50']:
        shot = f'{SAVE_PATH}/{model_name}_{i}_shot.json'
        try:

            data.append(
                {   
                    # 'kobest_boolq (macro_f1)': 0.35461338879835325
                    f"{k} ({_get_metric_name(v)['metric']})": _get_metric_name(v)['value']
                    for k, v in json.load(open(shot))['results'].items()
                }
            )
        except FileNotFoundError:
            pass
    df = pd.DataFrame(data, index=[model_name]).T
    return df

In [64]:
filelst

['yanolja_KoSOLAR-10.7B-v0.2',
 'yanolja_EEVE-Korean-Instruct-10.8B-v1.0',
 'aipart_chatjob_sft_from_Synatra-Mixtral-8x7B-v0.2',
 'upstage_SOLAR-10.7B-v1.0',
 'aipart_chatjob_sft_from_Synatra-Mixtral-8x7B-v0.1',
 'aipart_chatjob_sft_from_llama2_0.1_ALL_ver2',
 'mistralai_Mixtral-8x7B-v0.1',
 'OPEN-SOLAR-KO-10.7B',
 'llama_2_ko_7b',
 'open-llama-2-ko-7b']

In [62]:
SAVE_PATH = '../data/result'
filelst = get_filelst(SAVE_PATH)
preferred_order = {'aipart_chatjob_sft_from_llama2_0.1_ALL_ver2': 1, 
                   'aipart_chatjob_sft_from_Synatra-Mixtral-8x7B-v0.1': 2, 
                   'aipart_chatjob_sft_from_Synatra-Mixtral-8x7B-v0.2': 3, 
                   'llama_2_ko_7b': 4,
                   'open-llama-2-ko-7b': 5,
                   'upstage_SOLAR-10.7B-v1.0': 6,
                   'OPEN-SOLAR-KO-10.7B':7,
                   'mistralai_Mixtral-8x7B-v0.1': 8,
                   'yanolja_KoSOLAR-10.7B-v0.2': 9,
                   'yanolja_EEVE-Korean-Instruct-10.8B-v1.0':10}
ordered_list = sorted(filelst, key=lambda x: preferred_order[x])
temp_df = []
for file in ordered_list:
    df = get_df_klue(SAVE_PATH,model_name=file)
    temp_df.append(df)
all_df = pd.concat(temp_df,axis=1).T.reset_index()
all_df.rename(columns={"index":"name"},inplace=True)
all_df.to_csv("data.csv",index=False)

In [63]:
all_df

,name,kobest_boolq (macro_f1),kobest_copa (macro_f1),kobest_hellaswag (macro_f1),kobest_sentineg (macro_f1),kohatespeech (macro_f1),kohatespeech_apeach (macro_f1),kohatespeech_gen_bias (macro_f1),korunsmile (f1),nsmc (acc),pawsx_ko (acc)
0,yanolja_KoSOLAR-10.7B-v0.2,0.897134,0.736466,0.448120,0.429367,0.244705,0.510251,0.122301,0.383198,0.62522,0.5565
1,yanolja_EEVE-Korean-Instruct-10.8B-v1.0,0.918468,0.750659,0.486695,0.843542,0.150443,0.546269,0.242038,0.272436,0.76564,0.5860
2,aipart_chatjob_sft_from_Synatra-Mixtral-8x7B-v0.2,0.334282,0.487667,0.229223,0.533299,0.249713,0.337667,0.124535,0.237926,0.50368,0.4605
3,upstage_SOLAR-10.7B-v1.0,0.334282,0.578595,0.429190,0.331650,0.336987,0.337667,0.124535,0.434808,0.49732,0.4765
4,aipart_chatjob_sft_from_Synatra-Mixtral-8x7B-v0.1,0.354613,0.657732,0.449188,0.525259,0.477003,0.338516,0.124535,0.306483,0.56638,0.5390
5,aipart_chatjob_sft_from_llama2_0.1_ALL_ver2,0.748813,0.745854,0.395731,0.370676,0.296863,0.383731,0.165430,0.351791,0.53120,0.5350
6,mistralai_Mixtral-8x7B-v0.1,0.744271,0.663406,0.448753,0.613361,0.401101,0.353926,0.124535,0.357863,0.58316,0.5520
7,OPEN-SOLAR-KO-10.7B,0.854665,0.807557,0.501099,0.351700,0.261676,0.337667,0.124535,0.381959,0.53574,0.5390
8,llama_2_ko_7b,0.612147,0.759784,0.447951,0.351700,0.314636,0.346127,0.204651,0.290663,0.57942,0.5380
9,open-llama-2-ko-7b,0.341852,0.767270,0.478635,0.351700,0.318428,0.337667,0.127236,0.414869,0.55512,0.5190


In [48]:
import pandas as pd
import pygwalker as pyg
# df = pd.read_csv('./bike_sharing_dc.csv')
walker = pyg.walk(df,hideDataSourceConfig=True, vegaTheme='vega')

Box(children=(HTML(value='<div id="ifr-pyg-00061411ef97d86bnK34D5FYoLQ2kw8I" style="height: auto">\n    <head>…